## Transform Orders data - JSON String to JSON OBJECT
1. Pre-process the JSON string to fix the Data quality issues
1. Transform JSON string to JSON objects
1. Write transformed data to silver layer


In [0]:

select * from gizmobox.bronze.v_orders

In [0]:
create or replace temp view v_orders as
select regexp_replace(value, '"order_date": ([0-9]{4}-\\d{2}-\\d{2})', '"order_date": "\$1-"') as fixed_value from gizmobox.bronze.v_orders

#### Transform JSON string to JSON objects
1. Function : schema_of_json
1. Function : from_json

In [0]:
select 
schema_of_json(fixed_value) as schema,
fixed_value
 from v_orders
 limit 1;



In [0]:
select
from_json(fixed_value, 
'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as JSON_Value
from v_orders

#### Write transformed data to silver layer

In [0]:
create table if not exists gizmobox.silver.orders_json
as
select
from_json(fixed_value, 
'STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>') as JSON_Value
from v_orders

In [0]:
select * from gizmobox.silver.orders_json